# Basic Usage of FLIM Processing Library

This notebook demonstrates the basic usage of the FLIM Processing Library for analyzing fluorescence lifetime imaging microscopy (FLIM) data.

## Overview

The FLIM Processing Library provides tools for:
- Computing phasor coordinates from photon count data
- Performing curve fitting (RLD and LMA algorithms)
- Managing FLIM data with snapshot support
- Visualizing results with matplotlib

This example shows how to work with synthetic FLIM data to understand the core functionality.

## Setup

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from flim_processing import (
    FlimParams,
    DisplaySettings,
    ProcessingSettings,
    DataManager,
    PhasorComputer,
    FittingEngine
)

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)

## Generate Synthetic FLIM Data

Let's create synthetic photon count data representing a fluorescence decay curve. We'll simulate a simple exponential decay with some noise.

In [ ]:
def generate_synthetic_decay(height, width, time_bins, tau, period, noise_level=0.1):
    """
    Generate synthetic FLIM data with exponential decay.
    
    Args:
        height, width: Spatial dimensions
        time_bins: Number of time bins
        tau: Fluorescence lifetime in nanoseconds
        period: Laser period in nanoseconds
        noise_level: Relative noise level (default: 0.1)
    
    Returns:
        Photon count array of shape (height, width, time_bins)
    """
    # Time axis
    t = np.linspace(0, period, time_bins)
    
    # Generate exponential decay
    # I(t) = A * exp(-t/tau) + Z (background)
    A = 1000  # Amplitude
    Z = 50    # Background
    decay = A * np.exp(-t / tau) + Z
    
    # Create spatial variation in lifetime
    photon_count = np.zeros((height, width, time_bins))
    for i in range(height):
        for j in range(width):
            # Add spatial variation (±20% lifetime variation)
            tau_local = tau * (1 + 0.2 * np.sin(i / height * 2 * np.pi) * np.cos(j / width * 2 * np.pi))
            local_decay = A * np.exp(-t / tau_local) + Z
            
            # Add Poisson noise
            photon_count[i, j, :] = np.random.poisson(local_decay)
    
    return photon_count

# Generate synthetic data
height, width = 64, 64
time_bins = 256
tau = 2.5  # nanoseconds
period = 12.5  # nanoseconds (80 MHz laser)

photon_count = generate_synthetic_decay(height, width, time_bins, tau, period)

print(f"Generated photon count data with shape: {photon_count.shape}")
print(f"Data range: [{photon_count.min():.0f}, {photon_count.max():.0f}]")

## Visualize the Decay Curve

Let's look at a typical decay curve from the center pixel:

In [ ]:
# Extract decay curve from center pixel
center_decay = photon_count[height//2, width//2, :]

# Time axis
t = np.linspace(0, period, time_bins)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(t, center_decay, 'b-', linewidth=2, label='Measured decay')
plt.xlabel('Time (ns)', fontsize=12)
plt.ylabel('Photon Count', fontsize=12)
plt.title('Fluorescence Decay Curve (Center Pixel)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

## Configure FLIM Parameters

Set up the parameters for FLIM analysis:

In [ ]:
# Create FLIM parameters
flim_params = FlimParams(
    period=period,
    fit_start=10,  # Skip initial bins to avoid IRF effects
    fit_end=200    # Use most of the decay curve
)

print(f"FLIM Parameters:")
print(f"  Period: {flim_params.period} ns")
print(f"  Fit range: [{flim_params.fit_start}, {flim_params.fit_end})")

## Compute Phasor Coordinates

Phasor analysis represents lifetime data in the frequency domain using (g, s) coordinates:

In [ ]:
# Compute phasor coordinates
phasor = PhasorComputer.compute_phasor(photon_count, flim_params)

print(f"Phasor array shape: {phasor.shape}")
print(f"g range: [{np.nanmin(phasor[..., 0]):.3f}, {np.nanmax(phasor[..., 0]):.3f}]")
print(f"s range: [{np.nanmin(phasor[..., 1]):.3f}, {np.nanmax(phasor[..., 1]):.3f}]")

## Visualize Phasor Plot

The phasor plot shows the distribution of lifetime values in the frequency domain:

In [ ]:
# Extract g and s coordinates
g = phasor[..., 0].flatten()
s = phasor[..., 1].flatten()

# Remove NaN values for plotting
valid = ~(np.isnan(g) | np.isnan(s))
g_valid = g[valid]
s_valid = s[valid]

# Create phasor plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 2D histogram
h = ax1.hist2d(g_valid, s_valid, bins=50, cmap='viridis')
ax1.set_xlabel('g', fontsize=12)
ax1.set_ylabel('s', fontsize=12)
ax1.set_title('Phasor Plot (2D Histogram)', fontsize=14)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
plt.colorbar(h[3], ax=ax1, label='Count')

# Draw universal circle
theta = np.linspace(0, np.pi, 100)
circle_g = 0.5 + 0.5 * np.cos(theta)
circle_s = 0.5 * np.sin(theta)
ax1.plot(circle_g, circle_s, 'r--', linewidth=2, label='Universal circle')
ax1.legend(fontsize=10)

# Scatter plot (subsample for clarity)
subsample = np.random.choice(len(g_valid), min(1000, len(g_valid)), replace=False)
ax2.scatter(g_valid[subsample], s_valid[subsample], alpha=0.5, s=10)
ax2.plot(circle_g, circle_s, 'r--', linewidth=2, label='Universal circle')
ax2.set_xlabel('g', fontsize=12)
ax2.set_ylabel('s', fontsize=12)
ax2.set_title('Phasor Plot (Scatter)', fontsize=14)
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

## Perform Curve Fitting

Now let's fit the decay curves using both RLD (Rapid Lifetime Determination) and LMA (Levenberg-Marquardt) algorithms:

In [ ]:
# Compute RLD fitting
rld_result = FittingEngine.compute_rld(photon_count, flim_params)

print("RLD Fitting Results:")
print(f"  Tau shape: {rld_result.tau.shape}")
print(f"  Tau range: [{np.nanmin(rld_result.tau):.3f}, {np.nanmax(rld_result.tau):.3f}] ns")
print(f"  Chi-squared range: [{np.nanmin(rld_result.chisq):.3f}, {np.nanmax(rld_result.chisq):.3f}]")

# Compute LMA fitting (using RLD as initialization)
lma_result = FittingEngine.compute_lma(photon_count, flim_params)

print("\nLMA Fitting Results:")
print(f"  Parameters shape: {lma_result.param.shape}")
print(f"  Tau range: [{np.nanmin(lma_result.param[..., 2]):.3f}, {np.nanmax(lma_result.param[..., 2]):.3f}] ns")
print(f"  Chi-squared range: [{np.nanmin(lma_result.chisq):.3f}, {np.nanmax(lma_result.chisq):.3f}]")

## Visualize Lifetime Maps

Display the fitted lifetime values as spatial maps:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# RLD lifetime map
im1 = axes[0].imshow(rld_result.tau, cmap='viridis', vmin=2.0, vmax=3.0)
axes[0].set_title('RLD Lifetime Map', fontsize=14)
axes[0].set_xlabel('X (pixels)', fontsize=11)
axes[0].set_ylabel('Y (pixels)', fontsize=11)
plt.colorbar(im1, ax=axes[0], label='Lifetime (ns)')

# LMA lifetime map
im2 = axes[1].imshow(lma_result.param[..., 2], cmap='viridis', vmin=2.0, vmax=3.0)
axes[1].set_title('LMA Lifetime Map', fontsize=14)
axes[1].set_xlabel('X (pixels)', fontsize=11)
axes[1].set_ylabel('Y (pixels)', fontsize=11)
plt.colorbar(im2, ax=axes[1], label='Lifetime (ns)')

# Chi-squared map (LMA)
im3 = axes[2].imshow(lma_result.chisq, cmap='hot', vmin=0, vmax=5)
axes[2].set_title('LMA Chi-Squared Map', fontsize=14)
axes[2].set_xlabel('X (pixels)', fontsize=11)
axes[2].set_ylabel('Y (pixels)', fontsize=11)
plt.colorbar(im3, ax=axes[2], label='Chi-squared')

plt.tight_layout()
plt.show()

## Compare RLD and LMA Results

Let's compare the lifetime estimates from both algorithms:

In [ ]:
# Flatten arrays for comparison
rld_tau_flat = rld_result.tau.flatten()
lma_tau_flat = lma_result.param[..., 2].flatten()

# Remove NaN values
valid = ~(np.isnan(rld_tau_flat) | np.isnan(lma_tau_flat))
rld_tau_valid = rld_tau_flat[valid]
lma_tau_valid = lma_tau_flat[valid]

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
ax1.scatter(rld_tau_valid, lma_tau_valid, alpha=0.3, s=10)
ax1.plot([2, 3], [2, 3], 'r--', linewidth=2, label='y=x')
ax1.set_xlabel('RLD Lifetime (ns)', fontsize=12)
ax1.set_ylabel('LMA Lifetime (ns)', fontsize=12)
ax1.set_title('RLD vs LMA Lifetime Estimates', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)
ax1.set_aspect('equal')

# Histogram of differences
diff = lma_tau_valid - rld_tau_valid
ax2.hist(diff, bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(0, color='r', linestyle='--', linewidth=2, label='Zero difference')
ax2.set_xlabel('LMA - RLD (ns)', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_title('Difference Distribution', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()

print(f"Mean difference (LMA - RLD): {np.mean(diff):.4f} ns")
print(f"Std difference: {np.std(diff):.4f} ns")

## Using DataManager

The DataManager class helps organize FLIM data with snapshot support:

In [ ]:
# Create a DataManager
data_manager = DataManager(
    shape=(height, width, time_bins),
    dtype=photon_count.dtype,
    delta_mode=False
)

# Add the data as an element
data_manager.add_element(seqno=0, frame=photon_count)

print(f"Frame count: {data_manager.get_frame_count()}")

# Create a snapshot
data_manager.snapshot()
print(f"Frame count after snapshot: {data_manager.get_frame_count()}")

# Retrieve the snapshot
snapshot_data = data_manager.get_photon_count(0)
print(f"Retrieved snapshot shape: {snapshot_data.shape}")
print(f"Data matches original: {np.allclose(snapshot_data, photon_count)}")

## Summary

This notebook demonstrated the basic usage of the FLIM Processing Library:

1. **Data Generation**: Created synthetic FLIM data with exponential decay
2. **Phasor Analysis**: Computed phasor coordinates and visualized the phasor plot
3. **Curve Fitting**: Applied RLD and LMA algorithms to extract lifetime values
4. **Visualization**: Created lifetime maps and compared fitting results
5. **Data Management**: Used DataManager for organizing FLIM data

The library provides a clean, framework-agnostic API that works seamlessly with NumPy and matplotlib, making it ideal for Jupyter notebook workflows.